# HW1: Frame-Level Speech Recognition

In this homework, you will be working with MFCC data consisting of 15 features at each time step/frame. Your model should be able to recognize the phoneme occured in that frame.

# README


**Best Model Accuracy: 85.8%**

How to run the model: For Google Colab, simply clicking the right corner Runtime-> Run all will run the model.

**Model Structure:**
6 hidden layer, each has width 2048.
Activation: ReLu
Regularization:Dropout(0.1), BatchNorm1d
Learning rate:0.001
Context:30
Batch size:2048
optimizer: AdamW
Epoch:30
Parameter size:19.0464M

**Experiments:**

Hidden Layer size selection: I started with using multiple hidden layer with small width such as 512 and 256, and it turns out that the accuracy is only 78%, so I decide to try using wider hidden layer. Changing to 2048 increase the parameter size significantly, and the accuracy raise to 82%.

Context: I adjust my context for several experiments(using size 15,30, and 45). Size 30 gives the best result.

optimizer: I first using Adam as my optimizer. After discussing with TA and using AdamW instead, my accuracy increase from 82 to 83.

Epochs:

I start with using 10 epochs only, and it limits the amount of accuracy that the model can reach. After increase the epoch size into 30, My model increase to 85.8 from 83. 


For information related to all the experiments that I run, please visit:
WandB: https://wandb.ai/shenyix/hw1p2






# Library

In [ ]:
!pip install torchsummaryX wandb --quiet

     |████████████████████████████████| 1.9 MB 2.1 MB/s 
     |████████████████████████████████| 162 kB 90.1 MB/s 
     |████████████████████████████████| 182 kB 81.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 162 kB 82.6 MB/s 
     |████████████████████████████████| 158 kB 67.6 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 75.1 MB/s 
     |████████████████████████████████| 157 kB 83.1 MB/s 
     |████████████████████████████████| 157 kB 76.8 MB/s 
     |████████████████████████████████| 157 kB 82.7 MB/s 
     |████████████████████████████████| 157 kB 71.7 MB/s 
     |████████████████████████████████| 157 kB 68.3 MB/s 
     |████████████████████████████████| 156 kB 52.3 MB/s 


In [ ]:
import torch
import numpy as np
from torchsummaryX import summary
import sklearn
import gc
import zipfile
import pandas as pd
from tqdm.auto import tqdm
import os
import datetime
import wandb
import sklearn.metrics
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device: ", device)

Device:  cuda


In [ ]:
### If you are using colab, you can import google drive to save model checkpoints in a folder
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
### PHONEME LIST
PHONEMES = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']

# Kaggle

This section contains code that helps you install kaggle's API, creating kaggle.json with you username and API key details. Make sure to input those in the given code to ensure you can download data from the competition successfully.

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"shenyix","key":"c6455229fb59a0255b82591d44d4a323"}') 
    # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 59 kB 1.7 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=023c26bff9f173512ae8879eceb3838d662b8da24e3be04af56c683d5e4f3c8a
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
# commands to download data from kaggle

!kaggle competitions download -c 11-785-f22-hw1p2
!mkdir '/content/data'

!unzip -qo '11-785-f22-hw1p2.zip' -d '/content/data'

 99% 2.12G/2.13G [00:08<00:00, 318MB/s]
100% 2.13G/2.13G [00:08<00:00, 280MB/s]


In [ ]:
#out=os.listdir("/content/data/dev-clean/transcript/7850-111771-006.npy")
out=np.load("/content/data/dev-clean/mfcc/0084-121123-006.npy")
out2=np.load("/content/data/dev-clean/transcript/0084-121123-005.npy")
print(out.shape)
print(out2.shape)

(561, 15)
(1597,)


# Dataset

This section covers the dataset/dataloader class for speech data. You will have to spend time writing code to create this class successfully. We have given you a lot of comments guiding you on what code to write at each stage, from top to bottom of the class. Please try and take your time figuring this out, as it will immensely help in creating dataset/dataloader classes for future homeworks.

Before running the following cells, please take some time to analyse the structure of data. Try loading a single MFCC and its transcipt, print out the shapes and print out the values. Do the transcripts look like phonemes?

In [ ]:
# Dataset class to load train and validation data

class AudioDataset(torch.utils.data.Dataset):

    def __init__(self, data_path, context, offset=0, partition= "train", limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path

        if partition=="train":
          path="/content/data/train-clean-100"
        else:
          path="/content/data/dev-clean"

        # TODO: MFCC directory - use partition to acces train/dev directories from kaggle data
        self.mfcc_dir =path+"/mfcc"
      
        
        self.transcript_dir = path+"/transcript"
        # TODO: Transcripts directory - use partition to acces train/dev directories from kaggle data

        mfcc_names = (os.listdir(self.mfcc_dir))
        #print(len(mfcc_names))
        mfcc_names.sort()
         # TODO: List files in sefl.mfcc_dir_dir using os.listdir in sorted order, optionally subset using limit to slice the number of files you load
        transcript_names =(os.listdir(self.transcript_dir))
        #print(len(transcript_names))
        transcript_names.sort()
        assert len(mfcc_names) == len(transcript_names) # Making sure that we have the same no. of mfcc and transcripts

        self.mfccs, self.transcripts = [], []

       
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
        #   Load a single mfcc
            mfcc = np.load(self.mfcc_dir+"/"+mfcc_names[i])
            mfcc=(mfcc-mfcc.mean(axis=0))/mfcc.std(axis=0)
        #   Optionally do Cepstral Normalization of mfcc
        #   Load the corresponding transcript
            transcript = np.load(self.transcript_dir+"/"+transcript_names[i]) 
            transcript=transcript[(transcript != "<sos>") & (transcript != "<eos>")]
                
            # Remove [SOS] and [EOS] from the transcript (Is there an efficient way to do this 
            # without traversing through the transcript?)
        #   Append each mfcc to self.mfcc, transcript to self.transcript
            self.mfccs.append(mfcc)
            self.transcripts.append(transcript)

        

        # NOTE:
        # Each mfcc is of shape T1 x 15, T2 x 15, ...
        # Each transcript is of shape (T1+2) x 15, (T2+2) x 15 before removing [SOS] and [EOS]

        # TODO: Concatenate all mfccs in self.mfccs such that the final shape is T x 15 (Where T = T1 + T2 + ...) 
        self.mfccs = np.concatenate(self.mfccs)
        print(len(self.mfccs))
        
        self.length = len(self.mfccs)

        # TODO: Concatenate all transcripts in self.transcripts such that the final shape is (T,) meaning, each time step has one phoneme output
        self.transcripts = np.concatenate(self.transcripts)
        # Hint: Use numpy to concatenate

        # Take some time to think about what we have done. self.mfcc is an array of the format (Frames x Features). Our goal is to recognize phonemes of each frame
        # From hw0, you will be knowing what context is. We can introduce context by padding zeros on top and bottom of self.mfcc
        self.mfccs=np.vstack((np.vstack((np.zeros((self.context,15)),self.mfccs)),np.zeros((self.context,15))))

        # These are the available phonemes in the transcript
        self.phonemes = [
            'SIL',   'AA',    'AE',    'AH',    'AO',    'AW',    'AY',  
            'B',     'CH',    'D',     'DH',    'EH',    'ER',    'EY',
            'F',     'G',     'HH',    'IH',    'IY',    'JH',    'K',
            'L',     'M',     'N',     'NG',    'OW',    'OY',    'P',
            'R',     'S',     'SH',    'T',     'TH',    'UH',    'UW',
            'V',     'W',     'Y',     'Z',     'ZH',    '<sos>', '<eos>']
        # But the neural network cannot predict strings as such. Instead we map these phonemes to integers

        def helper(x):
          return self.phonemes.index(x)

        self.transcripts = np.vectorize(helper)(self.transcripts)
         # TODO: Map the phonemes to their corresponding list indexes in self.phonemes
        # Now, if an element in self.transcript is 0, it means that it is 'SIL' (as per the above example)

        # Length of the dataset is now the length of concatenated mfccs/transcripts
        

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind:ind+self.context*2+1] # TODO: Based on context and offset, return a frame at given index with context frames to the left, and right.
        # After slicing, you get an array of shape 2*context+1 x 15. But our MLP needs 1d data and not 2d.
        frames = frames.flatten()# TODO: Flatten to get 1d data

        frames = torch.FloatTensor(frames) # Convert to tensors
        phoneme = torch.tensor(self.transcripts[ind])       

        return frames, phoneme

In [ ]:
class AudioTestDataset(torch.utils.data.Dataset):
  

    # TODO: Create a test dataset class similar to the previous class but you dont have transcripts for this
    # Imp: Read the mfccs in sorted order, do NOT shuffle the data here or in your dataloader.
    def __init__(self, data_path, context, offset=0, limit=-1): # Feel free to add more arguments

        self.context = context
        self.offset = offset
        self.data_path = data_path
      
         
       
        self.mfcc_dir =data_path+"/test-clean/mfcc"
        mfcc_names = (os.listdir(self.mfcc_dir))
        mfcc_names.sort() 
        self.mfccs = []

       
        # Iterate through mfccs and transcripts
        for i in range(0, len(mfcc_names)):
    
            mfcc = np.load(self.mfcc_dir+"/"+mfcc_names[i])
            mfcc=(mfcc-mfcc.mean(axis=0))/mfcc.std(axis=0)
            self.mfccs.append(mfcc)
        
       
        self.mfccs = np.concatenate(self.mfccs)
        self.length = len(self.mfccs)
        self.mfccs=np.vstack((np.vstack((np.zeros((self.context,15)),self.mfccs)),np.zeros((self.context,15))))

        

    def __len__(self):
        return self.length

    def __getitem__(self, ind):
        
        frames = self.mfccs[ind:ind+self.context*2+1]
        frames = frames.flatten()
        frames = torch.FloatTensor(frames)
  
        return frames
       

# Parameters Configuration

Storing your parameters and hyperparameters in a single configuration dictionary makes it easier to keep track of them during each experiment. It can also be used with weights and biases to log your parameters for each experiment and keep track of them across multiple experiments. 

In [ ]:
config = {
    'epochs': 30,
    'batch_size' : 2048,
    'context' : 35,
    'learning_rate' : 0.001,
    'architecture' : 'high-cutoff'
    # Add more as you need them - e.g dropout values, weight decay, scheduler parameters
}

# Create Datasets

In [ ]:
train_data =AudioDataset(data_path="/content/data", context=config["context"], offset=0, partition= "train")

val_data =AudioDataset(data_path="/content/data", context=config["context"], offset=0, partition= "val")

# TODO: Create a dataset object using the AudioTestDataset class for the test data 

36191134
1937496


In [ ]:
len(train_data)
len(val_data)

1937496

In [ ]:
test_data = AudioTestDataset(data_path="/content/data", context=config["context"], offset=0) 

In [ ]:
# Define dataloaders for train, val and test datasets
# Dataloaders will yield a batch of frames and phonemes of given batch_size at every iteration
train_loader = torch.utils.data.DataLoader(train_data, num_workers= 4,
                                           batch_size=config['batch_size'], pin_memory= True,
                                           shuffle= True)

val_loader = torch.utils.data.DataLoader(val_data, num_workers= 2,
                                         batch_size=config['batch_size'], pin_memory= True,
                                         shuffle= False)

test_loader = torch.utils.data.DataLoader(test_data, num_workers= 2, 
                                          batch_size=config['batch_size'], pin_memory= True, 
                                          shuffle= False)


print("Batch size: ", config['batch_size'])
print("Context: ", config['context'])
print("Input size: ", (2*config['context']+1)*15)
print("Output symbols: ", len(PHONEMES))

print("Train dataset samples = {}, batches = {}".format(train_data.__len__(), len(train_loader)))
print("Validation dataset samples = {}, batches = {}".format(val_data.__len__(), len(val_loader)))
print("Test dataset samples = {}, batches = {}".format(test_data.__len__(), len(test_loader)))

Batch size:  2048
Context:  35
Input size:  1065
Output symbols:  42
Train dataset samples = 36191134, batches = 17672
Validation dataset samples = 1937496, batches = 947
Test dataset samples = 1943253, batches = 949


In [ ]:
# Testing code to check if your data loaders are working
for i, data in enumerate(train_loader):
    frames, phoneme = data
    print(frames.shape, phoneme.shape)
    break

torch.Size([2048, 1065]) torch.Size([2048])


# Network Architecture


This section defines your network architecture for the homework. We have given you a sample architecture that can easily clear the very low cutoff for the early submission deadline.

In [ ]:
# This architecture will make you cross the very low cutoff
# However, you need to run a lot of experiments to cross the medium or high cutoff
class Network(torch.nn.Module):

    def __init__(self, context):

        super(Network, self).__init__()

        input_size = (2*context + 1) * 15 #Why is this the case?: because with context frame
        output_size = 40 #Why? because we have 
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_size,2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1), 

            torch.nn.Linear(2048,2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),

            torch.nn.Linear(2048,2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),

            torch.nn.Linear(2048,2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),

            torch.nn.Linear(2048,2048),
            torch.nn.BatchNorm1d(2048),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.1),

            

            torch.nn.Linear(2048,output_size),

            )
  
    def forward(self, x):
        out = self.model(x)

        return out

# Define Model, Loss Function and Optimizer

Here we define the model, loss function, optimizer and optionally a learning rate scheduler. 

In [ ]:
input_size = 15*(2*config['context'] + 1)
model = Network(config['context']).to(device)
frames,phoneme = next(iter(train_loader))
# Check number of parameters of your network - Remember, you are limited to 20 million parameters for HW1 (including ensembles)
summary(model, frames.to(device))

                         Kernel Shape  Output Shape     Params  Mult-Adds
Layer                                                                    
0_model.Linear_0         [1065, 2048]  [2048, 2048]  2.183168M   2.18112M
1_model.BatchNorm1d_1          [2048]  [2048, 2048]     4.096k     2.048k
2_model.ReLU_2                      -  [2048, 2048]          -          -
3_model.Dropout_3                   -  [2048, 2048]          -          -
4_model.Linear_4         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
5_model.ReLU_5                      -  [2048, 2048]          -          -
6_model.Dropout_6                   -  [2048, 2048]          -          -
7_model.Linear_7         [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
8_model.ReLU_8                      -  [2048, 2048]          -          -
9_model.Dropout_9                   -  [2048, 2048]          -          -
10_model.Linear_10       [2048, 2048]  [2048, 2048]  4.196352M  4.194304M
11_model.BatchNorm1d_11        [2048] 

/usr/local/lib/python3.7/dist-packages/torchsummaryX/torchsummaryX.py:101: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_sum = df.sum()


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_model.Linear_0,"[1065, 2048]","[2048, 2048]",2183168.0,2181120.0
1_model.BatchNorm1d_1,[2048],"[2048, 2048]",4096.0,2048.0
2_model.ReLU_2,-,"[2048, 2048]",NaN,NaN
3_model.Dropout_3,-,"[2048, 2048]",NaN,NaN
4_model.Linear_4,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
5_model.ReLU_5,-,"[2048, 2048]",NaN,NaN
6_model.Dropout_6,-,"[2048, 2048]",NaN,NaN
7_model.Linear_7,"[2048, 2048]","[2048, 2048]",4196352.0,4194304.0
8_model.ReLU_8,-,"[2048, 2048]",NaN,NaN


In [ ]:
criterion = torch.nn.CrossEntropyLoss() #Defining Loss function 
optimizer = torch.optim.AdamW(model.parameters(), lr=config['learning_rate']) #Defining Optimizer
# Recommended : Define Scheduler for Learning Rate, including but not limited to StepLR, MultiStepLR, CosineAnnealingLR, ReduceLROnPlateau, etc. 
# You can refer to Pytorch documentation for more information on how to use them.

# Training and Validation Functions

This section covers the training, and validation functions for each epoch of running your experiment with a given model architecture. The code has been provided to you, but we recommend going through the comments to understand the workflow to enable you to write these loops for future HWs.

In [ ]:
torch.cuda.empty_cache()
gc.collect()

60

In [ ]:
def train(model, optimizer, criterion, dataloader):

    model.train()
    train_loss = 0.0 #Monitoring Loss
    scheduler =torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[25,30], gamma=0.1)
    
    for iter, (mfccs, phonemes) in enumerate(dataloader):

        ### Move Data to Device (Ideally GPU)
        mfccs = mfccs.to(device)
        phonemes = phonemes.to(device)

        ### Forward Propagation
        logits = model(mfccs)

        ### Loss Calculation
        loss = criterion(logits, phonemes)
        train_loss += loss.item()

        ### Initialize Gradients
        optimizer.zero_grad()

        ### Backward Propagation
        loss.backward()

        ### Gradient Descent
        optimizer.step()
    scheduler.step()
  
    train_loss /= len(dataloader)
    return train_loss

In [ ]:
def eval(model, dataloader):

    model.eval() # set model in evaluation mode

    phone_true_list = []
    phone_pred_list = []

    for i, data in enumerate(dataloader):

        frames, phonemes = data
        ### Move data to device (ideally GPU)
        frames, phonemes = frames.to(device), phonemes.to(device) 

        with torch.inference_mode(): # makes sure that there are no gradients computed as we are not training the model now
            ### Forward Propagation
            logits = model(frames)

        ### Get Predictions
        predicted_phonemes = torch.argmax(logits, dim=1)
        
        ### Store Pred and True Labels
        phone_pred_list.extend(predicted_phonemes.cpu().tolist())
        phone_true_list.extend(phonemes.cpu().tolist())
        
        # Do you think we need loss.backward() and optimizer.step() here?
    
        del frames, phonemes, logits
        torch.cuda.empty_cache()

    ### Calculate Accuracy
    accuracy = sklearn.metrics.accuracy_score(phone_pred_list, phone_true_list) 
    return accuracy*100

# Weights and Biases Setup

This section is to enable logging metrics and files with Weights and Biases. Please refer to wandb documentationa and recitation 0 that covers the use of weights and biases for logging, hyperparameter tuning and monitoring your runs for your homeworks. Using this tool makes it very easy to show results when submitting your code and models for homeworks, and also extremely useful for study groups to organize and run ablations under a single team in wandb. 

We have written code for you to make use of it out of the box, so that you start using wandb for all your HWs from the beginning.

In [ ]:
wandb.login(key="6b14f00e0515c5740d7e07042926608efdc6e526") #API Key is in your wandb account, under settings (wandb.ai/settings)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shenyix. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Create your wandb run
run = wandb.init(
    name = "slack-run-2", ### Wandb creates random run names if you skip this field, we recommend you give useful names
    reinit=True, ### Allows reinitalizing runs when you re-run this cell
    project="hw1p2", ### Project should be created in your wandb account 
    config=config ### Wandb Config for your run
)

In [ ]:
### Save your model architecture as a string with str(model) 
model_arch = str(model)

### Save it in a txt file 
arch_file = open("model_arch.txt", "w")
file_write = arch_file.write(model_arch)
arch_file.close()

### log it in your wandb run with wandb.save()
wandb.save('model_arch.txt')

['/content/wandb/run-20221001_041302-2ymq7p3c/files/model_arch.txt']

# Experiment

Now, it is time to finally run your ablations! Have fun!

In [ ]:
# Iterate over number of epochs to train and evaluate your model
torch.cuda.empty_cache()

best_acc = 0.0 ### Monitor best accuracy in your run

for epoch in range(config['epochs']):
    print("\nEpoch {}/{}".format(epoch+1, config['epochs']))

    train_loss = train(model, optimizer, criterion, train_loader)
    accuracy = eval(model, val_loader)

    print("\tTrain Loss: {:.4f}".format(train_loss))
    print("\tValidation Accuracy: {:.2f}%".format(accuracy))


    ### Log metrics at each epoch in your run - Optionally, you can log at each batch inside train/eval functions (explore wandb documentation/wandb recitation)
    wandb.log({"train loss": train_loss, "validation accuracy": accuracy})

    ### Save checkpoint if accuracy is better than your current best
    if accuracy >= best_acc:

      ### Save checkpoint with information you want
      torch.save({'epoch': epoch,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict(),
              'loss': train_loss,
              'acc': accuracy}, 
        './model_checkpoint.pth')
      
      ### Save checkpoint in wandb
      wandb.save('checkpoint.pth')

    # Is your training time very high? Look into mixed precision training if your GPU (Tesla T4, V100, etc) can make use of it 
    # Refer - https://pytorch.org/docs/stable/notes/amp_examples.html

### Finish your wandb run
run.finish()


Epoch 1/30
	Train Loss: 0.6271
	Validation Accuracy: 82.74%

Epoch 2/30
	Train Loss: 0.4468
	Validation Accuracy: 84.39%

Epoch 3/30
	Train Loss: 0.3895
	Validation Accuracy: 84.84%

Epoch 4/30
	Train Loss: 0.3562
	Validation Accuracy: 85.12%

Epoch 5/30
	Train Loss: 0.3343
	Validation Accuracy: 85.24%

Epoch 6/30
	Train Loss: 0.3191
	Validation Accuracy: 85.27%

Epoch 7/30
	Train Loss: 0.3081
	Validation Accuracy: 85.35%

Epoch 8/30
	Train Loss: 0.2997
	Validation Accuracy: 85.41%

Epoch 9/30
	Train Loss: 0.2931
	Validation Accuracy: 85.44%

Epoch 10/30
	Train Loss: 0.2879
	Validation Accuracy: 85.44%

Epoch 11/30
	Train Loss: 0.2834
	Validation Accuracy: 85.45%

Epoch 12/30
	Train Loss: 0.2800
	Validation Accuracy: 85.45%

Epoch 13/30
	Train Loss: 0.2770
	Validation Accuracy: 85.45%

Epoch 14/30
	Train Loss: 0.2742
	Validation Accuracy: 85.46%

Epoch 15/30
	Train Loss: 0.2719
	Validation Accuracy: 85.50%

Epoch 16/30
	Train Loss: 0.2699
	Validation Accuracy: 85.50%

Epoch 17/30
	Tra

train loss,█▅▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation accuracy,▁▅▆▇▇▇▇▇▇▇▇▇▇▇████████████████
train loss,0.25372
validation accuracy,85.65948


# Testing and submission to Kaggle

Before we get to the following code, make sure to see the format of submission given in *random_submission.csv*. Once you have done so, it is time to fill the following function to complete your inference on test data. Refer the eval function from previous cells to get an idea of how to go about completing this function.

In [ ]:
def test(model, test_loader):
  ### What you call for model to perform inference?
  model.eval()

  ### List to store predicted phonemes of test data
  test_predictions = []

  ### Which mode do you need to avoid gradients?
  with torch.no_grad():

      for i, frames in enumerate(tqdm(test_loader)):

          frames = frames.float().to(device)             
          
          output = model(frames)

          ### Get most likely predicted phoneme with argmax
          predicted_phonemes = torch.argmax(output, dim=1)
        
          ### Store Pred and True Labels
          test_predictions.extend(predicted_phonemes.cpu().tolist())
          
          ### How do you store predicted_phonemes with test_predictions? Hint, look at eval 
          
  return test_predictions

In [ ]:
predictions = test(model, test_loader)

  0%|          | 0/949 [00:00<?, ?it/s]

In [ ]:
### Create CSV file with predictions
with open("./submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(predictions)):
        f.write("{},{}\n".format(i, predictions[i]))

In [ ]:
### Submit to kaggle competition using kaggle API
!kaggle competitions submit -c 11-785-f22-hw1p2 -f ./submission.csv -m "Test Submission"

100% 18.6M/18.6M [00:00<00:00, 43.7MB/s]
Successfully submitted to Frame-Level Speech Recognition